# Model detail

Using this notebook you can find information about individual models regarding:
 - Structure and parameters
 - Evaluation
 - Display training history
 - Display activation maps for individual images
 
 ! Ensemble model is limited to evaluation only

Display available models

In [ ]:
import sys

import tf_utils
from utils import *
from visual_utils import explain

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model

from padding_generator import generators

import matplotlib.pyplot as plt
import pandas as pd
import math

from pandas.core.groupby.generic import DataFrameGroupBy as agg

In [ ]:
available_models()

Select model of your choice by name

Model name corresponds to ```MODEL_FOLDER/model_{model_name}```

In [3]:
model_name = 'resnet_101'

PATHS = data_paths(model_name, create_if_missing=False)

Load model to evaluate it, check its structure or display CAM

In [ ]:
model = load_model(PATHS['best'], compile=True) 

Evaluate model for individual images and for studies

In [ ]:
batch_size = 64 #reduce if memory is unsifficient

set_type = SetType.TEST #TEST, VALID, TRAIN

# Generator codes: 
# b - no augmentation
# 1 - Modest aug
# 2 - Strong aug
# 
# suffix e for equalization, eg: 1e
# suffix g for gauss noise, eg: 2g
gen_code = '1'
gen = generators[gen_code]

e = study_eval(model, set_type, generator = gen, batch_size = batch_size, aggregation=agg.mean) # aggregation mean or max

print(('Individual images: accuracy: {:.4f}, kappa: {:.4f}\n' + 
      'Study: accuracy: {:.4f}, kappa: {:.4f}').format(*e))

Model summary

In [4]:
print(get_note(PATHS))

Resnet-101, input shape 224, aug 1, Nadam 1-e4, bn_m 0.1, full preactivation, batch size 64
--------------------
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
zero_padding_1 (ZeroPadding2D)  (None, 230, 230, 1)  0           input[0][0]                      
__________________________________________________________________________________________________
conv_first (Conv2D)             (None, 112, 112, 64) 3200        zero_padding_1[0][0]             
__________________________________________________________________________________________________
bn_first (BatchNormalization)   (None, 112, 112, 64) 256         conv_first[0]

Plot model structure

In [ ]:
#Large models might be plotted too small. Adjust dpi if labels are unreadable. 
#Since the JN might be unsuitable for plot browsing, you can export it using to_file arg
plot_model(model, 
           show_shapes=True,
           dpi=150,
           #to_file='model.png',
          )

Display model training progress over epochs 

In [ ]:
data=get_log(PATHS['log'])
metrics = ['cohen_kappa']

fig, axes = plt.subplots(1, len(metrics))
fig.set_size_inches(16, 5)

for metric, ax in zip(metrics, axes if len(metrics) > 1 else [axes]):
    ax.grid()
    ax.set_title(model_name + ' : ' + metric)
    ax.set_ylabel(metric)
    ax.set_xlabel('epoch')
    ax.plot(data.index[1:], data[metric][1:], label='train')
    ax.plot(data.index[1:], data['val_'+metric][1:], label='validation')
    ax.legend();

Display activation map using Grad cam

Highlighted areas shows places that lead for prediction of abnormality

In [ ]:
flow = flow_from_csv(SetType.TEST, shuffle=True, batch_size=1, input_shape=model, generator=generators['b'])

Display results one by one

In [ ]:
sample = flow.next()
label = sample[1]
img = sample[0]

cam, heat, pred = explain(img, model, 1, reduction = 0.2) 

print('LABEL: ' + str(label) + ', PRED: ' + str(pred))

fig, axs = plt.subplots(ncols=3, constrained_layout=False,figsize=(20, 7))

p0 = axs[0].imshow(heat)
axs[1].imshow(cam)
axs[2].imshow(np.squeeze(img), cmap='gray')

fig.colorbar(p0, ax=axs[0])

for ax in axs:
    ax.axis('off')